In [1]:
import numpy as np
import copy
import datetime

In [2]:
def Sigmoid(x):
    res = 1 / (1 + np.exp(-x))
    return res

def d_Sigmoid(x):
    y = Sigmoid(x) * (1 - Sigmoid(x))
    return y

def ReLU(x):
    x = np.maximum(0, x)
    return x
    
def d_ReLU(x):
    y=x.copy()
    y[y<=0] = 0
    y[y>0] = 1
    return y


def Linear(x):
    return x

def d_Linear(x):
    y = np.ones(shape=(x.shape), dtype = x.dtype)
    return y

activations_dict = {
'Sigmoid': [Sigmoid, d_Sigmoid],
'ReLU': [ReLU, d_ReLU], 
'Linear': [Linear, d_Linear]
}


## input layer

In [3]:

class input_layer:
    def __init__(s, size):
        s.size = size
        #s.values = np.zeros(shape=(size), dtype = float)
    
    def add_neuron(s):
        #add_v = np.zeros(shape=(n_of_neurons), dtype=float)
        #s.values = np.concatenate((s.values, add_v.T))
        s.size += 1

    def delete_neuron(s, neuron_number):
        s.size-=1
        
    def delete_new_prev_size(s):     
        return


    def add_new_prev_size(s):     
        return

        
    def print_info(s):
        print("IN LAYER\nsize: ", s.size)

    
    def print_pic(s):
        print_size = min(2, s.size)

        for i in range(print_size): 
            print("| |\t", end='')
        print("")
        for i in range(print_size):
            print(" v \t", end='')
        print("")
        for i in range(print_size):
            print(' @\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')

    def forward(s, x, to_print = False):
        s.values = x
        return x
        
    def forward_nu(s, x):
        return x

    def get_info(s):
        return s.size

## layers

In [41]:
class layer:
    def __init__(s, lr = 0.1, prev_size = 2, my_size=2, activation_type = "Sigmoid", weights = None, bias = None):
        s.lr = lr
        s.size = my_size
        s.prev_size = prev_size
        if (np.all(weights == None)):
            # s.weights = np.random.random((prev_size, s.size))
            s.w = np.random.random((s.size, prev_size))
        else:
            s.w = weights.copy()
            s.w = s.w.reshape((s.size, prev_size))
            
        if (np.all(bias == None)):
            s.b = np.random.random((s.size, 1))
        else:
            s.b = bias.copy()
            s.b = s.b.reshape((s.size, 1))
            
        s.activation_type = activation_type
        funcs = activations_dict.get(activation_type)
        s.activation_f = funcs[0]
        s.d_activation_f = funcs[1]
        
        s.Vdw = np.zeros(shape=(s.size, prev_size))
        s.Vdb = np.zeros(shape=(s.size, 1))
        
        s.Sdw = np.zeros(shape=(s.size, prev_size))
        s.Sdb = np.zeros(shape=(s.size, 1))
        s.t = 1
        s.epsilon = 1e-8


    def activate(s, x):
        return s.activation_f(x)
        
    def d_activate(s, x):
        return s.d_activation_f(x)  

    
    def forward(s, x, to_print = False):
        s.x = np.asarray(x)
        s.z = np.dot(s.w, s.x) + s.b
        
        if (to_print): 
            print('wT * x + b', s.z)

        s.a = s.activate(s.z)

        if (to_print): 
            print('s.a ',s.a)
            
        return s.a

    def backprop(s, da):
        s.dz = da * s.d_activate(s.z)
        s.da_ = np.dot(s.w.T, s.dz) 

        
        return s.da_
        
    def update_weights(s, optimizer = "SGD", beta1 = 0.9, beta2 = 0.999):
        
        m = s.x.shape[1]
        if (optimizer == "SGD"):
            
            s.dw = (1/m)*np.dot(s.dz, s.x.T)
            s.db = (1/m)*np.sum(s.dz, axis = 1, keepdims = True)
    
            s.w = s.w - s.lr * s.dw
            s.b = s.b - s.lr * s.db
            
        elif (optimizer=="SGDwM"):

            s.dw = (1/m)*np.dot(s.dz, s.x.T)
            s.db = (1/m)*np.sum(s.dz, axis = 1, keepdims = True)

            
            s.Vdw = beta1 * s.Vdw + (1 - beta1)*s.dw
            s.Vdb = beta1 * s.Vdb + (1 - beta1)*s.db
            
            s.w = s.w - s.lr * s.Vdw
            s.b = s.b - s.lr * s.Vdb
            
        elif (optimizer=="RMSProp"):

            s.dw = (1/m)*np.dot(s.dz, s.x.T)
            s.db = (1/m)*np.sum(s.dz, axis = 1, keepdims = True)

            s.Sdw = beta2 * s.Sdw + (1-beta2) * np.square(s.dw)
            s.Sdb = beta2 * s.Sdb + (1-beta2) * np.square(s.db)
            
            s.w = s.w - s.lr * s.dw / (np.sqrt(s.Sdw))
            s.b = s.b - s.lr * s.db / (np.sqrt(s.Sdb))
            
        elif (optimizer=="Adam"):

            s.dw = (1/m)*np.dot(s.dz, s.x.T)
            s.db = (1/m)*np.sum(s.dz, axis = 1, keepdims = True)
            
            s.Vdw = beta1 * s.Vdw + (1 - beta1)*s.dw
            s.Vdb = beta1 * s.Vdb + (1 - beta1)*s.db
            
            s.Sdw = beta2 * s.Sdw + (1-beta2) * np.square(s.dw)
            s.Sdb = beta2 * s.Sdb + (1-beta2) * np.square(s.db)

            # correct

            s.Vdw_ = s.Vdw / (1 - beta1**s.t)
            s.Vdb_ = s.Vdb / (1 - beta1**s.t)
            s.Sdw_ = s.Sdw / (1 - beta2**s.t)
            s.Sdb_ = s.Sdb / (1 - beta2**s.t)
            
            s.w = s.w - s.lr * s.Vdw_ / (np.sqrt(s.Sdw_) + s.epsilon)
            s.b = s.b - s.lr * s.Vdb_ / (np.sqrt(s.Sdb_)  + s.epsilon)
            s.t += 1
        else:
            print("NO SUCH OPTIMIZER!")
            return




   
            
        
    def print_info(s):
        print("my size: ", s.size)
        print("prev size: ", s.prev_size)
        print("w: ", s.w.shape, s.w, "\n")
        print("b: ", s.b.shape, s.b, "\n")

              
    def print_pic(s):
        print_size = min(2, s.size)
        print("\n╻...\nv...")
        for i in range(print_size):
            print('O\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')



    def correct_prev_size(s, new_prev_szie):
        dif = new_prev_szie - s.prev_size
        if dif > 0: # new prev is greater
            for i in range(dif):
                s.add_new_prev_size()
        elif dif < 0:
            dif*=-1
            for i in range(dif):
                s.delete_new_prev_size()
        s.prev_size = new_prev_szie


    def delete_neuron(s, neuron_number):
        s.w = np.delete(s.w, neuron_number, axis = 0)
        s.b = np.delete(s.b, neuron_number, axis = 0)
        s.size-=1
        
    def delete_new_prev_size(s):     
        s.w = np.delete(s.w, 0, axis = 1)
        s.prev_size -=1


    def add_neuron(s):     
        add_w = np.zeros(shape=(1, s.prev_size), dtype=float) + 0.1 # np.random.random((s.prev_size, n_of_neurons)) #

        s.w = np.concatenate((s.w, add_w), axis = 0)
        
        add_b = np.zeros(shape=(1, 1), dtype=float) + 0.1 
        s.b = np.concatenate((s.b, add_b))
        s.size+=1

    def add_new_prev_size(s):     
        add_w = np.zeros(shape=(1, s.size), dtype=float) + 0.1
        print(add_w)
        print(s.w)
        s.w = np.concatenate((s.w, add_w.T), axis = 1)
        s.prev_size += 1

    def get_info(s):
        return s.prev_size, s.size, s.w, s.b, s.activation_type, s.lr


        
        


## plastic nn

In [5]:

class plastic_nn:
    def __init__(s, optimizer = "SGD", beta = 0.9):

        opt_list = ['SGD', 'SGDwM', 'RMSProp', 'Adam']
        if optimizer in opt_list:
            s.optimizer = optimizer
        else:
            print('no such optimizer, available are: ')
            for each in opt_list:
                print(each)
            return
        
        s.layers = []
            
        s.n_of_layers = 0
        s.name = 'noname'
        s.optimizer = optimizer
        
        s.beta = beta
        pass

    def give_name(s, name):
        s.name = name
        
    def set_num_of_layers(s, num):
        s.n_of_layers = num
        
    def deep_copy(s):
        return copy.deepcopy(s)

    
    def forward(s, x, to_print = False):
        for lay in s.layers:
            x = lay.forward(x, to_print)
        s.last_result = x
        return s.last_result
        
    def forward_print(s, x, to_print = False):
        print('in: ',data)
        cnt = 0
        for lay in s.layers:
            x = lay.forward(x, to_print)
            print(cnt, ' ', x)
            cnt+=1
        s.last_result = x
        return s.last_result



    
    def backprop(s, correct):
        m = correct.shape[1]
        err = (s.last_result - correct) # a - y
        cnt = 0
        for lay in reversed(s.layers[1:]):
            #print(cnt)
            err = lay.backprop(err)
            cnt+=1

    def backprop_error(s, err):
        for lay in reversed(s.layers[1:]):
            err = lay.backprop(err)

    def update(s):
        i = 0
        for lay in reversed(s.layers[1:]):
            #print('layer idx: ', i)
            i+=1
            lay.update_weights(s.optimizer,s.beta)
            #print('\n')


    
    def learn_one(s, in_data, target_data):
        s.forward(in_data)
        s.backprop(target_data)
        s.update()   



    
    def append_one(s, new_layer, check = False):
        if check and s.n_of_layers!=0:
            last_layer_size = s.layers[-1].size
            if last_layer_size != new_layer.prev_size:
                print("size not match, layer ", s.n_of_layers)
                return
        s.layers.append(new_layer)
        s.n_of_layers+=1
        return

    def check_layers_sizes(s, check_layers):
        for i in range(1, len(check_layers)):
            if (check_layers[i-1].size != check_layers[i].prev_size):
                print("error between ", i-1, "and ", i)
                return False
        return True

    
    def append_layers(s, new_layers, to_print = False):
        test_layers = np.array([])
        if s.n_of_layers != 0: # if has layers
            test_layers = s.layers[-1] # get last layers
        
        test_layers = np.append(test_layers, new_layers) 
                
        if (s.check_layers_sizes(test_layers)):
            for lay in new_layers:
                s.append_one(lay)
            if (to_print):
                print("added LAYERS succesfully")
            return True
        else:
            if (to_print):
                print("ERROR adding layers, check info above")
            return False

    def add_layer_by_pos(s, pos, new_layer):
        if (pos <= 0 or pos > s.n_of_layers): # if input or more than 'to last'
            print("ERROR addning layer: invalid layer number!")
            if (pos == 0):
                print("input layer cannot be replaced by different layer")
            return
            
        if (pos == s.n_of_layers): # if add to the last
            s.append(new_layer)
            return
            
        if (new_layer.prev_size!=s.layers[pos-1].size):
            print("ERROR addning layer: invalid prev_size!")
            return 
            
        s.layers.insert(pos, new_layer)
        prev_size = new_layer.size
        
        # update next layer prev_size and w matrix
        next_lay = s.layers[pos+1]
        next_lay.correct_prev_size(prev_size)
        s.n_of_layers += 1

    def delete_layer_by_pos(s, pos):
        if (pos <= 0 or pos >= s.n_of_layers): # if input or more than 'to last'
            print("ERROR deleting layer: invalid layer number!")
            if (pos == 0):
                print("input layer cannot be deleted")
            return
        
        new_prev_size = s.layers[pos].prev_size 
        if (pos != s.n_of_layers-1): #if not last
            next_lay = s.layers[pos+1]
            next_lay.correct_prev_size(new_prev_size)

        del s.layers[pos]
        s.n_of_layers -= 1


    def add_neuron(s, layer_number, n_of_neurons = 1):
        if (layer_number < 0 or layer_number>= s.n_of_layers):
            print("ERROR addning neuron: invalid layer number!")
            return
        
        main_lay = s.layers[layer_number]  
        
        for i in range(n_of_neurons):
            main_lay.add_neuron()           
            if (layer_number+1 != s.n_of_layers): # if main is not last
                # update next layer prev_size and w matrix
                next_lay = s.layers[layer_number+1]
                next_lay.add_new_prev_size()
    
    def delete_neuron(s, layer_number, neuron_number):
        if (layer_number < 0 or layer_number>= s.n_of_layers):
            print("ERROR deleting neuron: invalid layer number!")
            return
            
        main_lay = s.layers[layer_number] 
        
        if (neuron_number >= main_lay.size):
            print("ERROR deleting neuron: invalid neuron number!")
            return

        main_lay.delete_neuron(neuron_number)
        if (layer_number+1 != s.n_of_layers): # if main is not last
                # update next layer prev_size and w matrix
                next_lay = s.layers[layer_number+1]
                next_lay.delete_new_prev_size()
        
        

    
    def print_info(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        for cnt in range(s.n_of_layers):
            print("#", cnt)
            s.layers[cnt].print_info()
            print("")
    
    def print_pic(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        cnt = 0
        for lay in s.layers:
            lay.print_pic()
            print("#", cnt, end='')
            cnt+=1
        print("\nOUT |#|\nOUT  v")

    
    def save(s, file_path):       
        f = open(file_path, "w").close()
        
        f = open(file_path, "a")       
        f.write("{}\n{}\n".format(s.name, s.n_of_layers))       
        input_layer_size = s.layers[0].get_info()
        f.write("{}\n".format(input_layer_size))

        for lay in s.layers[1:]:            
            prev_size, size, weights, bias, activation_type, lr = lay.get_info()
            f.write("{}\n".format(prev_size))
            f.write("{}\n".format(size))
            
            np.savetxt(f, weights)#, fmt='%f')
            np.savetxt(f, bias)#, fmt='%f')
            f.write("{}\n".format(activation_type))
            f.write("{}\n".format(lr))
        
        ct = datetime.datetime.now()


        f.write("\nct {}\n".format(ct))
        f.close()

        
    def load(s, file_path):
        s.layers = None
        s.layers = []
        s.n_of_layers = 0       
        layers = []
        f = open(file_path, "r")       
        name = f.readline().split()[0]
        total_n_of_layers = int(f.readline().split()[0])
        input_layer_size = int(f.readline().split()[0])

        in_layer = input_layer(input_layer_size)
        layers.append(in_layer)
        
        s.give_name(name)

        for i in range(total_n_of_layers-1):
            prev_size = int(f.readline().split()[0])
            size = int(f.readline().split()[0])
            weights = np.loadtxt(f, max_rows = size)
            bias = np.loadtxt(f, max_rows = size)
            activation_type = f.readline().split()[0]
            lr = float(f.readline().split()[0])

            layers.append(layer(lr = lr, prev_size = prev_size, my_size = size, 
                                activation_type = activation_type, weights = weights, bias = bias))
            
        s.append_layers(layers)             
        f.close()






## TESt


In [55]:
learning_rate_a = 0.0001
in_states = 3
h1_nodes = 7
h2_nodes = 7
out_actions = 5

a_type1 = 'ReLU'
a_type2 = 'Linear'

layers_net = [input_layer(in_states), 
layer(lr = learning_rate_a, prev_size = in_states, my_size=h1_nodes, activation_type=a_type1), 
#layer(lr = learning_rate_a, prev_size = h1_nodes, my_size=h2_nodes, activation_type=a_type1), 
layer(lr = learning_rate_a, prev_size = h2_nodes, my_size=out_actions, activation_type=a_type2)]

target_dqn = plastic_nn(optimizer="Adam")
target_dqn.append_layers(layers_net)

target_dqn.print_info()

NAME:  noname  ( 3 )
# 0
IN LAYER
size:  3

# 1
my size:  7
prev size:  3
w:  (7, 3) [[0.82357226 0.36806541 0.963958  ]
 [0.67236037 0.8268002  0.83520457]
 [0.89238573 0.17820765 0.27824368]
 [0.82694947 0.88210081 0.60924004]
 [0.23352039 0.62373086 0.10627872]
 [0.90312832 0.20565316 0.20256474]
 [0.68739479 0.83569961 0.79852254]] 

b:  (7, 1) [[0.71334704]
 [0.37216764]
 [0.15907392]
 [0.93981587]
 [0.56736907]
 [0.17185902]
 [0.69666584]] 


# 2
my size:  5
prev size:  7
w:  (5, 7) [[0.33804454 0.84830814 0.59171967 0.79746152 0.54274784 0.20071834
  0.27967253]
 [0.77878558 0.84134111 0.00159761 0.28391054 0.67829393 0.70043151
  0.85915624]
 [0.25571724 0.83205292 0.70184899 0.55026465 0.01989467 0.31692686
  0.814543  ]
 [0.11875776 0.77101622 0.26625152 0.47344489 0.60948683 0.0651628
  0.87701009]
 [0.95528718 0.33100788 0.30043053 0.50585956 0.83436261 0.41704653
  0.84836034]] 

b:  (5, 1) [[0.33184441]
 [0.48778173]
 [0.26301644]
 [0.74941903]
 [0.37323343]] 




In [56]:
add_layer = layer(lr = learning_rate_a, prev_size = h2_nodes, my_size=out_actions, activation_type=a_type2)
target_dqn.delete_layer_by_pos(1)
target_dqn.print_info()

NAME:  noname  ( 2 )
# 0
IN LAYER
size:  3

# 1
my size:  5
prev size:  3
w:  (5, 3) [[0.54274784 0.20071834 0.27967253]
 [0.67829393 0.70043151 0.85915624]
 [0.01989467 0.31692686 0.814543  ]
 [0.60948683 0.0651628  0.87701009]
 [0.83436261 0.41704653 0.84836034]] 

b:  (5, 1) [[0.33184441]
 [0.48778173]
 [0.26301644]
 [0.74941903]
 [0.37323343]] 


